#Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.
https://www.google.com/drive/

https://colab.research.google.com/

##(i) Here we import all necessary libraries
Please put all your files at a desired location. For example, under "/content/drive/MyDrive/Colab Notbooks/USC_BA_estimator/" folder and load our packages. 



In [ ]:
from google.colab import drive

drive.mount("/content/drive")
import numpy as np
import matplotlib.pyplot as plt
import os
import skimage as ski
import skimage.feature
import nibabel as nib
import nibabel.freesurfer.mghformat as mgh
import sys

# Path to our Jupyter notebook.
# sys.path.append(
#     os.path.join("/content/drive/MyDrive/test_brain_age/function", "function")
# )
sys.path.append(
    os.path.join("/content/drive/MyDrive/Colab Notebooks/test_brain_age", "function")
)
from scipy.ndimage import zoom
from glob import glob
import time
import datetime
import pandas as pd
from preprocess_parallel import processmgz
import savepath
import repadding
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
import modelcnn
import sklearn
import saliencymaps
import multiprocessing as mp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##(ii) Main function.


In [ ]:
def mainfunction(dir_test_brain_age, path_to_model_weights, path_to_subject_ids):
    # path to the folder (brain.mgz)
    id = pd.read_csv(path_to_subject_ids)
    path_of_mgz = dir_test_brain_age

    
    brains, coordinates = processmgz(path_of_mgz)
    time_split = time.ctime().split(" ")
    (
        brain_save_path,
        coordinates_save_path,
        coordinates_save_path_csv,
    ) = savepath.filename_brainnpy(path_of_mgz, time_split)

    # save the cropped brains
    np.save(brain_save_path, brains)

    # save information for repadding.
    np.save(coordinates_save_path, coordinates)
    df_coordinates = pd.DataFrame(coordinates)
    df_coordinates.columns = ["x", "y", "z"]
    df_coordinates["subj_id"] = id.iloc[:, 0]
    df_coordinates.to_csv(coordinates_save_path_csv)

    # Build model. 
    model = modelcnn.get_model(width=82, height=86, depth=100)

    # Load best weights.
    model.load_weights(path_to_model_weights)

    # make brain age predictions
    predictions = model.predict(brains) + 22

    # save the prediction results. Of note, the input subject ids should be saved as csv.
    
    BA_save_path_npy, BA_save_path_csv = savepath.filename_pred(path_of_mgz, time_split)
    df_BA = pd.DataFrame(predictions)
    df_BA.columns = ["BA"]
    df_BA["subj_id"] = id.iloc[:, 0]
    df_BA.to_csv(BA_save_path_csv)

    # Generate saliency mapes
    saliency_map_list = saliencymaps.smap(model, brains)

    # Repadding the saliency maps
    coordinates = np.load(coordinates_save_path)
    repadded_saliency_maps = repadding.repadding(saliency_map_list, coordinates)

    # Save the saliency maps
    smap_path = savepath.filename_smap(path_of_mgz, time_split)
    np.save(smap_path, repadded_saliency_maps)
    return df_BA, repadded_saliency_maps, brains


##(iii) Execute main function. 
"dir_test_brain_age" is the path to the folder which contains the mgz files. \\
"path_to_model_weights" is the path to the h5 file. \\
"path_to_subject_ids" is the path to the subject ids list. \\

The pre-processed mgz file will be saved under the path of "path_of_mgz". \\
The pre-processed mgz file is saved in this format "brain-year-month-day_hour-min-secs.npy". \\

The predicted BA will be saved as csv files under "dir_test_brain_age". \\
The csv file is saved in this format "BA-year-month-day_hour-min-sec.csv" \\

The processed saliency maps will be saved under "dir_test_brain_age."\\
The saliency maps will be saved in this format "Saliency_maps-year-month-day_hour-min-sec.csv" \\




In [ ]:
# path to the brain.mgz files
dir_test_brain_age = "/content/drive/MyDrive/Colab Notebooks/test_brain_age"

# path to the h5 file
path_to_model_weights = os.path.join(
    dir_test_brain_age, "function/model/saved-model-317-3.50.h5"
)

# path ot the subject ids file. Of note, this code only support csv files.
path_to_subject_ids = os.path.join(dir_test_brain_age, "subj_id.csv")

# the mainfunction will automatically save and output the predicted BAs and saliency maps.
predicted_BA, repadded_saliency_maps, brains = mainfunction(
    dir_test_brain_age, path_to_model_weights, path_to_subject_ids
)


1/1 [==============================] - 2s 2s/step


In [ ]:
brains.shape, predicted_BA

((2, 82, 86, 100, 1),
           BA     subj_id
 0  72.929840  141_S_0853
 1  83.862534  153_S_4172)